In [1]:
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

import pyperclip
import time
import datetime

def top_to_bottom(browser, i):
    start = time.time()
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(2.5)

        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            time.sleep(1)
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
        last_height = new_height  
        if i:
            if (time.time() - start) > i:
                break
                
def print_csv(f_name, name, contact, city):
    try:
        df = pd.DataFrame({"이름":name,
                           "연락처":contact,
                           "지역":city})
        df = df.fillna('null')
        numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
        int_values = []
        for item in numbers:
            if item.isdigit():
                int_values.append(int(item))
            else:
                int_values.append('없음')
        df.isnull().sum()
        df = df.fillna('null')
        numbers = df['연락처'].str.split('년').str.get(0).str[-4:]
        int_values = []
        for item in numbers:
            if item.isdigit():
                int_values.append(int(item))
            else:
                int_values.append('없음')
        df['출생연도'] = int_values
        df.columns = ['이름', '연락처', '지역', '출생연도']
        df['지역'] = df['지역'].str.split('이전 거주지').str.get(1).str.split('현재').str.get(0).fillna('표시할 장소 없음')
        
        if len(df[df['연락처'].str.contains('\+82')]) >= 1:
            df['연락처'] = df['연락처'].str.split('\+82 ').str.get(1).str[0:12].fillna('없음')
            df.index[df['연락처'].str.contains('1')]
            phone_num = df.index[df['연락처'].str.contains('1')]
            df.loc[phone_num, '연락처'] = "0" + df.loc[phone_num, '연락처']
        else:
            df['연락처'] = '없음'
        df.to_csv(f"{f_name}.csv", encoding='UTF-8')
    except Exception as ex:
        print(f'{len(name)} {len(contact)} {len(city)}')
        df = pd.DataFrame({"이름":name,
                           "연락처":contact,
                           "지역":city})

#open browser with secret mode
options = webdriver.ChromeOptions()
options.add_argument("--proxy-server=socks5://127.0.0.1:9150")
options.add_argument('window-size=1920x1080')
options.add_argument('disable-gpu')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument('--no-sandbox')
options.add_argument('no-sandox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--start-maximized')
options.add_argument('incognito')

service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service, options=options)

#open facebook
browser.get("https://www.facebook.com/")
time.sleep(2)

#log info, search keyword
USER = "shm8485@gmail.com"
PWD = "sohee8485"

#login
elem_id = browser.find_element("id", "email")
pyperclip.copy(USER)
elem_id.send_keys(Keys.CONTROL, "v")
time.sleep(2)

elem_pw = browser.find_element("id", "pass")
pyperclip.copy(PWD)
elem_pw.send_keys(Keys.CONTROL, "v")
elem_pw.send_keys("\n")
time.sleep(3)

#search
keyword = ['군포', '안양', '산본', '부동산', '경기도']
kw_city = None

df_name, df_contact, df_city = [], [], []

#loop with keywords
for kw in keyword:
    #search keyword in order
    browser.get(f"https://www.facebook.com/search/groups/?q={kw}")
    time.sleep(2)
    
    #except block alert
    try:
        elem_block = browser.find_element("xpath", "/html/body/div[3]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div")
        elem_block.click()
    except:
        pass
    try:
        elem_block = browser.find_element('xpath', '/html/body/div[5]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div')
        elem_block.click()
    except:
        pass
    time.sleep(1)


    #show only public groups
    elem_public = browser.find_element('xpath', '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]/div/div[9]/div[2]/div[2]/div/div[2]/div/input')
    elem_public.click()

    #city
    if kw_city:
        elem_city = browser.find_element("xpath", "/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]/div/div[9]/div[2]/div[1]/div/div/div/div/div/div")
        time.sleep(1)
        elem_city.click()
        elem_city = browser.find_element("xpath", '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]/div/div[9]/div[2]/div[1]/div/div/div/div/div/div/input')
        time.sleep(1)
        pyperclip.copy(kw_city)
        elem_city.send_keys(Keys.CONTROL, "v")
        time.sleep(1)         
        elem_city = browser.find_element('xpath', '/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[2]/div/div/div[1]/div[1]/div/ul/li[1]/div/div[1]/div/div/div')
        elem_city.click()
        time.sleep(1) 

    top_to_bottom(browser, 5)
    time.sleep(5)
    
    #get group links
    soup_groups = bs(browser.page_source, "html.parser")
    top_elements = soup_groups.find_all(class_="x1yztbdb")
    groups_href = []

    try:
        for t in top_elements:
            groups_href.append(t.find("a").attrs['href'])
    except Exception as ex:
        print(ex)
    
    #add 'members' at the end to open members page directly
    final_href = []
    for g in groups_href:
        final_href.append(g[:len(g)-11]+'members')
    
    #save all group names and file names
    group_name, f_name = [], []

    #loop with groups
    for i in final_href:
        #open the group page
        browser.get(i)
        
        #except block alert
        try:
            elem_block = browser.find_element("xpath", "/html/body/div[3]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div")
            elem_block.click()
        except:
            pass
        try:
            elem_block = browser.find_element('xpath', '/html/body/div[5]/div[1]/div/div[2]/div/div/div/div/div/div/div[1]/div/div[2]/div')
            elem_block.click()
        except:
            pass
        time.sleep(1)
        
        # scroll to bottom
        top_to_bottom(browser)
        time.sleep(1.5)
        
        #get pofile links
        soup_profile = bs(browser.page_source, "html.parser")
        h_list = soup_profile.find_all('h1')
        top_elements = soup_profile.find_all(class_="x1lq5wgf xgqcy7u x30kzoy x9jhf4c x1lliihq")
        #get group name
        for h in h_list:
            if h.text == '알림': continue
            elif h.text == '검색결과': continue
            else: g_name = h.text
        print(g_name)
        
        #get profile links
        profile_list = []
        profile_href = []
        for j in top_elements:
            profile_list.append(j.find("a").attrs['href'])

        for j in profile_list:
            profile_href.append('https://www.facebook.com/profile.php?id='+j[j.index('user/')+5:-1])
        
        #save elements
        name, contact, city = [], [], []
        error = []
        url = 'php?id='
        print(len(profile_href))
        
        #loop with profiles
        for j in profile_href:
            try:
                print(profile_href.index(j))
                #open the profile page
                browser.get(j)
                #is profile link with id or numbers
                temp = browser.current_url
                if url in temp:
                    link = j + '&sk='
                else:
                    link = temp[:temp.index('?')]
                #open 
                browser.get(link+"about_contact_and_basic_info")
                soup_info = bs(browser.page_source, "html.parser")
                soup_name = soup_info.find(class_="x78zum5 x15sbx0n x5oxk1f x1jxijyj xym1h4x xuy2c7u x1ltux0g xc9uqle")
                soup_info_contact = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
                t_cont = soup_info_contact.text
                if '+82 ' in t_cont:
                    contact.append(t_cont)
                    name.append(soup_name.find("h1").text)
                    browser.get(link+"about_places")
                    soup_info = bs(browser.page_source, "html.parser")
                    soup_info_city = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
                    city.append(soup_info_city.text)
                else: continue

            except:
                l_name = len(name)
                l_cont = len(contact)
                l_city = len(city)
                print(browser.current_url)
                error.append(browser.current_url)
                if l_name > l_cont:
                    contact.append('이전 거주지표시할 장소 없음')
                    try:
                        browser.get(link+"about_places")
                        soup_info = bs(browser.page_source, "html.parser")
                        soup_info_ct = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
                        city.append(soup_info_ct.text)
                    except:
                        city.append('')
                elif l_cont > l_city:
                    city.append('')
                else: 
                    continue
                                
        # get real time
        timestamp = datetime.datetime.now()
        t_stamp = timestamp.strftime("%Y-%m-%d_%H%M%S")
        if kw_city:
            t_filename = f'{kw}_{kw_city}_{t_stamp}'
        else:
            t_filename = f'{kw}_{t_stamp}'
        
        if len(name) == 0:
            group_name.append(g_name)
            f_name.append(' ')
            print(f'{g_name}: nothing in there')
            continue
        else:
            df_name.append(name)
            df_contact.append(contact)
            df_city.append(city)
            print_csv(t_filename, name, contact, city)
            print(f'{g_name}: {t_filename}')
            group_name.append(g_name)
            f_name.append(t_filename)
    
    timestamp = datetime.datetime.now()
    t_stamp = timestamp.strftime("%Y-%m-%d_%H%M%S")
    if kw_city:
        t_filename = f'{kw}_{kw_city}_filelist_{t_stamp}'
    else:
        t_filename = f'{kw}_filelist_{t_stamp}'

    print_csv(f'{kw} 총합', df_name, df_contact, df_city)

    df = pd.DataFrame({"그룹 이름":group_name,
                       "파일명":f_name})
    df.to_csv(f"{t_filename}.csv", encoding='UTF-8')


군포 의왕 산본 안양 무료나눔 교환 직거래
25
0
1
1
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
군포 의왕 산본 안양 무료나눔 교환 직거래: nothing in there
♡유튜브 협업 구인구직♡ (군포/안양/과천/안산/수원)
97
0
1
2
3
4
5
6
https://www.facebook.com/win0314271001/about_contact_and_basic_info
7
8
9
10
11
12
13
2
15
16
0
18
https://www.facebook.com/profile.php?id=100042073696218&sk=about_places
19
20
21
22
23
https://www.facebook.com/profile.php?id=100063190332651&sk=about_places
24
25
26
27
28
29
30
31
32
33
34
35
https://www.facebook.com/skincelllove/about_places
36
37
38
39
40
https://www.facebook.com/smiledrone.co.kr/about_places
41
42
https://www.facebook.com/Goldhome.TV/about_places
43
44
https://www.facebook.com/kpeepilates/about_places
45
46
47
48
49
50
51
52
1
54
55
56
57
58
59
60
61
62
63
https://www.facebook.com/happydday.shop/about_places
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
♡유튜브 협업 구인구직♡ (군포/안양/과천/안산/수원): 군포_2023-06-27_183306
군포,산본,안양,평촌,의왕,과천 이야기
84
0
1
2

93
94
95
96
안양1번가조이당구장: nothing in there
군포,산본,안양,평촌,의왕,과천 이야기
84
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
https://www.facebook.com/yeonseshintong/about_places
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
군포,산본,안양,평촌,의왕,과천 이야기: 안양_2023-06-28_003621
♡유튜브 협업 구인구직♡ (군포/안양/과천/안산/수원)
87
0
1
2
3
4
5
6
https://www.facebook.com/win0314271001/about_contact_and_basic_info
7
8
9
10
11
12
13
2
15
16
0
18
https://www.facebook.com/profile.php?id=100042073696218&sk=about_places
19
20
21
22
23
https://www.facebook.com/profile.php?id=100063190332651&sk=about_places
24
25
26
27
28
29
30
31
32
33
34
35
https://www.facebook.com/skincelllove/about_places
36
37
38
39
40
https://www.facebook.com/smiledrone.co.kr/about_places
41
42
https://www.facebook.com/Goldhome.TV/about_places
43
44
https://www.facebook.com/kpeepilates/about_places
45
46
47
48
49
50
51

83
84
85
86
87
88
89
90
91
92
93
(안산,시흥 사람들의 맛집&여행 모임): 여행_2023-06-28_061706
여행
90
0
1
2
3
1
5
6
7
https://www.facebook.com/worldkoreanheritage/about_contact_and_basic_info
8
https://www.facebook.com/Travelholic24/about_contact_and_basic_info
9
https://www.facebook.com/buanmasil/about_contact_and_basic_info
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
여행: 여행_2023-06-28_063743
관광?여행?볼거리?먹거리?즐길거리
99
0
1
2
3
4
3
4
7
8
https://www.facebook.com/profile.php?id=105437951527246&sk=about_contact_and_basic_info
9
10
11
12
13
14
15
16
17
https://www.facebook.com/goodcoop.co.kr/about_places
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div/div[1]/div/div[3]/div/div/div/div[1]/div[1]/div[1]/div/div[2]/div[1]/div[2]/div/div/div[2]/div/div[9]/div[2]/div[2]/div/div[2]/div/input"}
  (Session info: chrome=114.0.5735.134); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
Backtrace:
	GetHandleVerifier [0x005DA813+48355]
	(No symbol) [0x0056C4B1]
	(No symbol) [0x00475358]
	(No symbol) [0x004A09A5]
	(No symbol) [0x004A0B3B]
	(No symbol) [0x004CE232]
	(No symbol) [0x004BA784]
	(No symbol) [0x004CC922]
	(No symbol) [0x004BA536]
	(No symbol) [0x004982DC]
	(No symbol) [0x004993DD]
	GetHandleVerifier [0x0083AABD+2539405]
	GetHandleVerifier [0x0087A78F+2800735]
	GetHandleVerifier [0x0087456C+2775612]
	GetHandleVerifier [0x006651E0+616112]
	(No symbol) [0x00575F8C]
	(No symbol) [0x00572328]
	(No symbol) [0

In [4]:
from webdriver_manager.chrome import ChromeDriverManager

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

import pyperclip
import time
import datetime


In [5]:
options = webdriver.ChromeOptions()
options.add_argument("--proxy-server=socks5://127.0.0.1:9150")
options.add_argument('window-size=1920x1080')
options.add_argument('disable-gpu')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_argument('--no-sandbox')
options.add_argument('no-sandox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--start-maximized')
options.add_argument('incognito')

service = Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=service, options=options)

In [6]:
browser.get("https://www.facebook.com/jenya.belyakova.9/about")

In [17]:
link = browser.current_url[:-5]

browser.get(link+"about_contact_and_basic_info")
soup_info = bs(browser.page_source, "html.parser")
soup_name = soup_info.find(class_="x78zum5 x15sbx0n x5oxk1f x1jxijyj xym1h4x xuy2c7u x1ltux0g xc9uqle")
soup_info_contact = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
t_cont = soup_info_contact.text
if '+82 ' in t_cont:
    contact.append(t_cont)
    name.append(soup_name.find("h1").text)
    browser.get(link+"about_places")
    soup_info = bs(browser.page_source, "html.parser")
    soup_info_city = soup_info.find(class_="xyamay9 xqmdsaz x1gan7if x1swvt13")
    city.append(soup_info_city.text)


AttributeError: 'NoneType' object has no attribute 'text'

In [16]:
print('+82 'in t_cont)

True


In [22]:

        print(len(profile_href[:10]))

10


In [ ]:
import pandas as pd